# lecture 12. TF

In [2]:


import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense,Dropout
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

import os
import warnings

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
warnings.filterwarnings('ignore')



/tmp/ipykernel_21140/7591248.py:14: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
train_data = pd.read_csv('har_train.csv')
test_data = pd.read_csv('har_test.csv')

print(f'Shape of train data is: {train_data.shape}\nShape of test data is: {test_data.shape}')

Shape of train data is: (7352, 563)
Shape of test data is: (2947, 563)


In [4]:


x_train, y_train = train_data.iloc[:, :-2], train_data.iloc[:, -1:]
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_train.shape, y_train.shape



((7352, 561), (7352, 1))

In [5]:
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_test.shape, y_test.shape

((2947, 561), (2947, 1))

In [6]:


le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)



In [7]:


x_test.shape, y_test.shape, x_train.shape, y_train.shape



((2947, 561), (2947,), (7352, 561), (7352,))

In [8]:
scaling_data = MinMaxScaler()
x_train = scaling_data.fit_transform(x_train)
x_test = scaling_data.transform(x_test)

In [9]:
model = Sequential()

In [10]:
model.add(Dense(units=64,kernel_initializer='normal',activation='sigmoid',input_dim=x_train.shape[1]))

2024-05-31 16:19:31.510480: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 16:19:31.511122: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
model.add(Dropout(0.2))

In [12]:
model.add(Dense(units=6,kernel_initializer='normal',activation='softmax'))

In [13]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [14]:
history = model.fit(x_train, y_train, batch_size = 64, epochs= 10,validation_data = (x_test,y_test))

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4059 - loss: 1.4951 - val_accuracy: 0.7767 - val_loss: 1.0221
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.6959 - loss: 0.9422 - val_accuracy: 0.8558 - val_loss: 0.7142
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - accuracy: 0.8072 - loss: 0.6489 - val_accuracy: 0.8721 - val_loss: 0.5226
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.8471 - loss: 0.4868 - val_accuracy: 0.8819 - val_loss: 0.4198
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.8755 - loss: 0.3988 - val_accuracy: 0.8982 - val_loss: 0.3550
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.9000 - loss: 0.3326 - val_accuracy: 0.8989 - val_loss: 0.3132
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.9037 - loss: 0.2940 - val_accuracy: 0.9131 - val_loss: 0.2770
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.9070 - loss: 0.2641 - va

In [15]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)):
        model.add(layers.Dense(units = hp.Int('units' + str(i), min_value=32, max_value=512, step=32),
                               kernel_initializer= hp.Choice('initializer', ['uniform', 'normal']),
                               activation= hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])))
    model.add(layers.Dense(6, kernel_initializer= hp.Choice('initializer', ['uniform', 'normal']), activation='softmax'))
    model.add(
            Dropout(0.2))
    model.compile(
        optimizer = 'adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials= 5,
    executions_per_trial=3,
    directory='project', project_name = 'Human_activity_recognition')

tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 25, 'step': 1, 'sampling': 'linear'}
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
initializer (Choice)
{'default': 'uniform', 'conditions': [], 'values': ['uniform', 'normal'], 'ordered': False}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [16]:
tuner.search(x_train, y_train,
             epochs= 10,
             validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 32s]
val_accuracy: 0.7094219923019409

Best val_accuracy So Far: 0.7094219923019409
Total elapsed time: 00h 03m 04s


In [18]:


model.summary()
import tensorflow as tf
from tensorflow import keras

Callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
mo_fitt = model.fit(x_train,y_train, epochs=200, validation_data=(x_test,y_test), callbacks=Callback)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │        35,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,076 (426.08 KB)

 Trainable params: 36,358 (142.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 72,718 (284.06 KB)

Epoch 1/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.9317 - loss: 0.1891 - val_accuracy: 0.9345 - val_loss: 0.1875
Epoch 2/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.9457 - loss: 0.1658 - val_accuracy: 0.9172 - val_loss: 0.2019
Epoch 3/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.9453 - loss: 0.1540 - val_accuracy: 0.9420 - val_loss: 0.1579
Epoch 4/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.9517 - loss: 0.1309 - val_accuracy: 0.9304 - val_loss: 0.1705
Epoch 5/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.9580 - loss: 0.1216 - val_accuracy: 0.9274 - val_loss: 0.1717
Epoch 6/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.9572 - loss: 0.1184 - val_accuracy: 0.9348 - val_loss: 0.1639
Epoch 7/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.9644 - loss: 0.1020 - val_accuracy: 0.9430 - val_loss: 0.1495
Epoch 8/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.9665 - loss: 0

https://www.kaggle.com/code/fahadmehfoooz/human-activity-recognition-with-neural-networks